In [1]:
from keras.datasets import mnist
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

print(train_images.shape)
train_images=train_images.reshape(train_images.shape[0],train_images.shape[1]*train_images.shape[2])
print(train_images.shape)
test_images=test_images.reshape(test_images.shape[0],test_images.shape[1]*test_images.shape[2])
print(train_images.shape)

Using TensorFlow backend.


(60000, 28, 28)
(60000, 784)
(60000, 784)


In [2]:
# normalize training img and test img
train_images_normalised = train_images/255.0
test_images_normalised = test_images/255.0

In [3]:
print(np.mean(train_images_normalised))
train_images_normalised_copy=train_images_normalised.copy()
for i in range(train_images_normalised.shape[0]):
        for j in range(train_images_normalised.shape[1]):
            if (train_images_normalised[i][j] > 0.5):
                train_images_normalised_copy[i][j]=1
            else:
                train_images_normalised_copy[i][j]=0

print(train_images_normalised_copy.shape)

0.1306604762738429
(60000, 784)


In [4]:

test_images_normalised_copy=test_images_normalised.copy()
for i in range(test_images_normalised.shape[0]):
        for j in range(test_images_normalised.shape[1]):
            if (test_images_normalised[i][j] > 0.5):
                test_images_normalised_copy[i][j]=1
            else:
                test_images_normalised_copy[i][j]=0

print(test_images_normalised_copy.shape)

(10000, 784)


In [5]:
def plot_digit(some_digit):
    
    some_digit_image = some_digit.reshape(28,28)

    plt.imshow(some_digit_image, cmap = matplotlib.cm.binary, interpolation = "nearest")
    plt.axis("off")
    plt.show()

In [6]:

#print(train_images_normalised[0].shape)
#plot_digit(train_images_normalised[30])
#plot_digit(train_images_normalised_copy[30])


In [7]:
def find_edge(data):
    edge=[]
    sign=0
    b=0
    for i in range(0,28): 
        sign=0
        for j in range(0,28):
            if data[i][j]==0 and sign==1:
                edge.append(((i*28+j),(i*28+j-1)))
            if data[i][j]==0:
                sign=1
            else:
                sign=0
            if data[i][j]==1:
                b=b+1
    for j in range(0,28): 
        sign=0
        for i in range(0,28):
            if data[i][j]==0 and sign==1:
                edge.append(((i*28+j),((i-1)*28+j)))
            if data[i][j]==0:
                sign=1
            else:
                sign=0
    return edge,b

In [8]:
#print(train_images_normalised[30].shape)
#testlist,count_b=find_edge(train_images_normalised_copy[30].reshape(28,28))

#print(count_b)

In [9]:
res_train=np.zeros((train_images_normalised.shape[0],(train_images_normalised.shape[1]+1)))
res_test=np.zeros((test_images_normalised.shape[0],(test_images_normalised.shape[1]+1)))
print(res_train.shape)
print(res_test.shape)

(60000, 785)
(10000, 785)


In [10]:
for i in range (train_images_normalised_copy.shape[0]):
    class QuickUnion(object):
        id=[]
        count=0
    
        def __init__(self,n):
            self.count = n
            i=0
            while i<n:
                self.id.append(i)
                i+=1

        def connected(self,p,q):
            if self.find(p) == self.find(q):
                return True
            else:            
                return False
    
        def find(self,p):   
            while (p != self.id[p]):
                p = self.id[p]
            return p
    
        def union(self,p,q):
            idq = self.find(q)
            idp = self.find(p)
            if not self.connected(p,q):
                self.id[idp]=idq
                self.count -=1
            

    testlist,count_b=find_edge(train_images_normalised_copy[i].reshape(28,28))
    #print(count_b)
    qf = QuickUnion(28*28)
    for k in testlist:
        p =  k[0]
        q =  k[1]
        qf.union(p,q)
    
    
    #print("res:",qf.count-count_b)
    res_train[i]=np.append(train_images_normalised[i],(qf.count-count_b)/3)
    testlist=[]
    count_b=0
    qf=0

In [11]:
#Using union find to add new features
for i in range (test_images_normalised_copy.shape[0]):
    class QuickUnion(object):
        id=[]
        count=0
    
        def __init__(self,n):
            self.count = n
            i=0
            while i<n:
                self.id.append(i)
                i+=1

        def connected(self,p,q):
            if self.find(p) == self.find(q):
                return True
            else:            
                return False
    
        def find(self,p):   
            while (p != self.id[p]):
                p = self.id[p]
            return p
    
        def union(self,p,q):
            idq = self.find(q)
            idp = self.find(p)
            if not self.connected(p,q):
                self.id[idp]=idq
                self.count -=1
            

    testlist,count_b=find_edge(test_images_normalised_copy[i].reshape(28,28))
    #print(count_b)
    qf = QuickUnion(28*28)
    for k in testlist:
        p =  k[0]
        q =  k[1]
        qf.union(p,q)
    
    
    #print("res:",qf.count-count_b)
    res_test[i]=np.append(test_images_normalised[i],(qf.count-count_b)/3)
    testlist=[]
    count_b=0
    qf=0

In [12]:
from keras.utils import np_utils
from keras.models import Sequential 
from keras.layers import Dense, Activation 
input_dim = 785 #28*28 +1
output_dim = nb_classes = 10 
Y_train = np_utils.to_categorical(train_labels, nb_classes) 
Y_test = np_utils.to_categorical(test_labels, nb_classes)

In [13]:
model = Sequential() 
model.add(Dense(output_dim, input_dim=input_dim, activation='softmax')) 
batch_size = 128 
nb_epoch = 70

In [14]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy']) 
history = model.fit(res_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,verbose=1, validation_data=(res_test, Y_test)) 
score = model.evaluate(res_test, Y_test, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

C:\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 60000 samples, validate on 10000 samples
Epoch 1/70
60000/60000 [==============================] - 3s 53us/step - loss: 1.2723 - acc: 0.7056 - val_loss: 0.8069 - val_acc: 0.8359
Epoch 2/70
60000/60000 [==============================] - 1s 19us/step - loss: 0.7091 - acc: 0.8433 - val_loss: 0.6026 - val_acc: 0.8634
Epoch 3/70
60000/60000 [==============================] - 1s 19us/step - loss: 0.5803 - acc: 0.8616 - val_loss: 0.5205 - val_acc: 0.8757
Epoch 4/70
60000/60000 [==============================] - 1s 19us/step - loss: 0.5181 - acc: 0.8722 - val_loss: 0.4742 - val_acc: 0.8831
Epoch 5/70
60000/60000 [==============================] - 1s 19us/step - loss: 0.4798 - acc: 0.8783 - val_loss: 0.4433 - val_acc: 0.8890
Epoch 6/70
60000/60000 [==============================] - 1s 19us/step - loss: 0.4533 - acc: 0.8836 - val_loss: 0.4212 - val_acc: 0.8931
Epoch 7/70
60000/60000 [==============================] - 1s 19us/step - loss: 0.4335 - acc: 0.8872 - val_loss: 0.4045 - val_acc

60000/60000 [==============================] - 1s 20us/step - loss: 0.2775 - acc: 0.9244 - val_loss: 0.2695 - val_acc: 0.9272
Epoch 61/70
60000/60000 [==============================] - 1s 20us/step - loss: 0.2768 - acc: 0.9243 - val_loss: 0.2686 - val_acc: 0.9273
Epoch 62/70
60000/60000 [==============================] - 1s 20us/step - loss: 0.2761 - acc: 0.9245 - val_loss: 0.2681 - val_acc: 0.9277
Epoch 63/70
60000/60000 [==============================] - 1s 20us/step - loss: 0.2753 - acc: 0.9250 - val_loss: 0.2674 - val_acc: 0.9279
Epoch 64/70
60000/60000 [==============================] - 1s 19us/step - loss: 0.2746 - acc: 0.9250 - val_loss: 0.2671 - val_acc: 0.9280
Epoch 65/70
60000/60000 [==============================] - 1s 19us/step - loss: 0.2740 - acc: 0.9253 - val_loss: 0.2665 - val_acc: 0.9281
Epoch 66/70
60000/60000 [==============================] - 1s 20us/step - loss: 0.2733 - acc: 0.9254 - val_loss: 0.2660 - val_acc: 0.9287
Epoch 67/70
60000/60000 [=====================